# mix-SQP demo with different low-rank matrix approximations

In this example, we illustrate how the QR and singular value decompositions of the likelihood matrix can be used to speed up the SQP algorithm. Compare these results to Fig. 2 of the paper.

## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP_time.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [13]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(x,gridmult = 1.425);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm a few times to precompile the relevant functions.

In [5]:
out = mixSQP_time(L,lowrank = "svd");
out = mixSQP_time(L,lowrank = "qr");
out = mixSQP_time(L,lowrank = "nothing");

Next, run the mix-SQP solver again with the SVD and QR approximations to the likelihood matrix, and with no approximation. The approximation tolerance is set very low, to `1e-10`.

In [6]:
@time outSVD = mixSQP_time(L,lowrank = "svd",pqrtol = 1e-10);
@time outQR  = mixSQP_time(L,lowrank = "qr",pqrtol = 1e-10);
@time out    = mixSQP_time(L,lowrank = "nothing");

  0.399308 seconds (19.58 k allocations: 311.511 MiB, 12.59% gc time)
  0.251480 seconds (21.74 k allocations: 321.207 MiB, 14.53% gc time)
  0.279777 seconds (19.88 k allocations: 643.426 MiB, 16.34% gc time)


You should see a slight improvement in the computation time with the QR and SVD approximations. And the solutions using the low-rank approximations are still very close to the solution without an approximation:

In [7]:
(rel_error(L,out[1],outSVD[1]), rel_error(L,out[1],outQR[1]))

(8.30002733209767e-12, 0.0)

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [8]:
srand(1);
x = normtmixdatasim(round(Int,1e6));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [9]:
sd = autoselectmixsd(x,gridmult = 1.0735);
L  = normlikmatrix(x,sd = sd);
size(L)

(1000000, 100)

## Fit mixture model using SQP algorithm 

As before, let's run the mix-SQP solver with the SVD and QR approximations, and with no approximation.

In [10]:
@time outSVD = mixSQP_time(L,lowrank = "svd",pqrtol = 1e-10);
@time outQR  = mixSQP_time(L,lowrank = "qr",pqrtol = 1e-10);
@time out    = mixSQP_time(L,lowrank = "nothing");

 12.198485 seconds (58.85 k allocations: 8.535 GiB, 21.69% gc time)
 10.693275 seconds (65.11 k allocations: 8.634 GiB, 18.24% gc time)
 66.649927 seconds (66.26 k allocations: 57.209 GiB, 9.77% gc time)


In the larger data set, the QR and SVD approximations yield much larger improvements in computation time, which is not surprising because the matrix-vector operations (particularly in computing the gradient and Hessian) dominate the overall computation time.

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here.

In [11]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.0


In [12]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
